# Setup

In [1]:
import pandas as pd
import json
import re

# Read Dataset

In [2]:
df = pd.read_csv('product_info_modified.csv').drop(columns=['Unnamed: 0'])

In [4]:
df.head()

,product_name,price,product_weight,nutritional_elements
0,Skinless Chicken Breast,$4.55,240g,"{""Attributes"": ""Per Serving (100g)"", ""Energy"":..."
1,Seng Choon Eggs - Farm Fresh,$3.65,550g,"{""Attributes"": ""Per Serving"", ""Energy"": ""68kca..."
2,Chew's Fresh Eggs with Vitamin E,$3.65,600g,"{""Attributes"": ""Per Serving (50g)"", ""Energy"": ..."
3,Dasoon Premium Fresh Eggs,$5.10,900g,"{""Attributes"": ""Per Serving (60g)"", ""Energy"": ..."
4,Nature's Wonders Baked Nuts - Almonds,$5.80,200g,"{""Attributes"": ""Per Serving (30g)"", ""Energy"": ..."


In [ ]:
# Parse json to dict objects
df['nutritional_elements_dict'] = df['nutritional_elements'].apply(lambda x: json.loads(x))

#### Extract numerical characters from `price` and `product_weight`

In [29]:
df = (
    df
    .assign(
        price_num = lambda x: x['price'].apply(lambda x: float(x.replace('$', ''))),
        product_weight_num = lambda x: x['product_weight'].str.extract('(\d+)', expand=False)
    )
)

## Extract relevant nutritional information

In [113]:
def parse_nutritional_value(
        nutrition_facts: dict,
        key: str):
    """
    Parses the desired nutritional value from the dictionary of nutritional facts

    :param nutrition_facts: dict to parse
    :param key: key to parse
    :return: serving size in numerical format (e.g., 100)
    """

    if key in nutrition_facts:
        to_parse = nutrition_facts[key]
        if key == "Attributes":
            match = re.search(r'\((\d+)', to_parse)
            if match:
                return match.group(1)
        else:
            return re.findall(r'([-+]?\d*\.?\d+)', to_parse)[0]
    return None

In [118]:
df = df.assign(
    serving_size = lambda x: x['nutritional_elements_dict'].apply(lambda x: parse_nutritional_value(x, "Attributes")),
    calories = lambda x: x['nutritional_elements_dict'].apply(lambda x: parse_nutritional_value(x, "Energy")),
    protein = lambda x: x['nutritional_elements_dict'].apply(lambda x: parse_nutritional_value(x, "Protein")),
    carbohydrate = lambda x: x['nutritional_elements_dict'].apply(lambda x: parse_nutritional_value(x, "Carbohydrate")),
    sugars = lambda x: x['nutritional_elements_dict'].apply(lambda x: parse_nutritional_value(x, "Sugars"))
)

#### Normalize price by serving size

In [120]:
# Drop rows with missing service size
df = df.dropna(subset=['serving_size'])

In [127]:
# Get the number of servings per product, then find the price per serving
df = (
    df
    .assign(
        num_servings = lambda x: round(x['product_weight_num'].astype(float) / x['serving_size'].astype(float), 1),
        price_per_serving = lambda x: round(x['price_num'] / x['num_servings'], 2)
    )
)

#### Remove products without calories, protein values or price_per_serving

In [130]:
df = df.dropna(subset=['calories', 'protein', 'price_per_serving'])

#### Normalize to 24g of protein 
We use Optimum Nutrition's Gold Standard Whey Protein as a reference point, which is a very popular nutrition brand that typically caters towards bodybuilding.

In [18]:
df.head(5)

,product_name,price,product_weight,nutritional_elements,serving_size,price_num,product_weight_num,calories,protein,carbohydrate,sugars,num_servings,price_per_serving,num_servings_24g_protein,price_per_24g_protein
0,Skinless Chicken Breast,$4.55,240g,"{""Attributes"": ""Per Serving (100g)"", ""Energy"":...",100,4.55,240,165.0,31.0,NaN,NaN,2.4,1.90,0.774194,1.47
1,Chew's Fresh Eggs with Vitamin E,$3.65,600g,"{""Attributes"": ""Per Serving (50g)"", ""Energy"": ...",50,3.65,600,70.0,6.6,1.4,0.0,12.0,0.30,3.636364,1.09
2,Dasoon Premium Fresh Eggs,$5.10,900g,"{""Attributes"": ""Per Serving (60g)"", ""Energy"": ...",60,5.10,900,75.0,7.7,70.0,0.0,15.0,0.34,3.116883,1.06
3,Nature's Wonders Baked Nuts - Almonds,$5.80,200g,"{""Attributes"": ""Per Serving (30g)"", ""Energy"": ...",30,5.80,200,150.0,7.7,13.0,0.7,6.7,0.87,3.116883,2.71
4,FairPrice Baked Almonds,$10.65,400g,"{""Attributes"": ""Per Serving (30g)"", ""Energy"": ...",30,10.65,400,212.0,7.0,9.2,1.5,13.3,0.80,3.428571,2.74


In [20]:
WHEY_PROTEIN = 24

df = df.assign(
    num_servings_24g_protein = lambda x: WHEY_PROTEIN / x['protein'],
    price_per_24g_protein = lambda x: round(x['price_per_serving'] * x['num_servings_24g_protein'], 2),
    calories_per_24g_protein = lambda x: round(x['calories'] * x['num_servings_24g_protein'], 2),
    carbs_per_24g_protein = lambda x: round(x['carbohydrate'] * x['num_servings_24g_protein'], 2),
    sugar_per_24g_protein = lambda x: round(x['sugars'] * x['num_servings_24g_protein'], 2)
)

#### Drop dict column to store as csv

In [133]:
(
    df
    .drop(
        columns=['nutritional_elements_dict']
    )
    .to_csv('product_info_cleaned.csv')
)